In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats
import statsmodels.stats.api as sms
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import kstest
from statsmodels.stats.diagnostic import het_white
df = pd.read_csv('NoOutliersDataset.csv', index_col= [0])

numerical = ["CO2 Emissions", "AFF", "Government Effectiveness", "Individuals using the Internet", "Life Expectancy", "Renewable Energy Consumption"]
categorical = ["Country", "Income Group", "Region"]

numerical_without_emissions = [x for x in numerical if x != "CO2 Emissions"]

df

In [ ]:
#the following function calculates a linear regression and tests its Gauss-Markov Theorem assumptions
#we will not be using train-test-split due to the low number of records
def Linear_Regression(df, dependent_variable, independent_variables, alpha, log_model = False):
    num_df = pd.DataFrame(data = df, columns = independent_variables)
    X = sm.add_constant(num_df)
    if log_model == True:
        model = sm.OLS(np.log(df[dependent_variable]), X)
    else:
        model = sm.OLS(df[dependent_variable], X)
    results = model.fit()
    residuals = results.resid

    #creating the test variables 

    err_mean_var = scipy.stats.ttest_1samp(residuals,0)
    corr_var = durbin_watson(residuals)
    hom_var_bpg = sms.het_breuschpagan(residuals, results.model.exog)
    hom_var_white = het_white(residuals, results.model.exog)
    norm_var = kstest(residuals, 'norm')

    #GMT assumptions

    err_mean_test = "The expected value of residuals is 0 ✅" if (err_mean_var[1] > alpha) else "Residuals have a mean different than 0 ❌"
    corr_test = "Errors are not correlated ✅" if (corr_var >= 1.5 and corr_var <= 2.5) else "Errors are correlated ❌"
    hom_test = "Errors are homoscedastic ✅" if (hom_var_bpg[1] > alpha or hom_var_white[1] > alpha) else "Heteroskedasticity is present ❌"
    norm_test = "Errors follow a normal distribution ✅" if(norm_var[1] > alpha) else "Errors do not follow a normal distribution ❌"

    #Results
    print("\n" + str(results.summary()) + "\n ---------------------------------------------------------------------------------")

    if (results.f_pvalue < alpha):
        print("A linear regression between the dependent variable CO2 Emissions and the independent variable/s " + str(independent_variables)+ 
        " resulted in a statistically significant R2 of " + str(round(results.rsquared,4)*100) + " %, with a p-value of " + str(results.f_pvalue))
    else:
        print("A linear regression between the dependent variable CO2 Emissions and the independent variable/s " + str(independent_variables)+ 
        " resulted in a statistically insignificant R2 of " + str(round(results.rsquared,4)*100) + " %, with a p-value of " + str(results.f_pvalue))

    print("\n🟡🟡🟡🟡🟡🟡🟡🟡🟡 Gauss Markov Theorem assumptions 🟡🟡🟡🟡🟡🟡🟡🟡🟡")
    print("\n" + err_mean_test + "\nT-test Statistic: " + str(err_mean_var[0]) + "\nP-value: " + str(err_mean_var[0]) + 
        "\n-------------------------------------------------------")
    print(" \n" + corr_test + "\nDurbin-Watson test: " + str(durbin_watson(residuals)) + 
        "\n-------------------------------------------------------" )
    print(" \n" + hom_test + "\nBreusch Pagan Godfrey: " + str(hom_var_bpg[0]) +  "\nP-value: " +  str(hom_var_bpg[1]) + 
    "\nWhite Test: " + str(hom_var_white[0]) + "\nP-value: " + str(hom_var_white[1])+ "\n-------------------------------------------------------")
    print(" \n" + norm_test + "\nKolmogorov Smirnov test: " + str(norm_var[0])+ "\nP-value: " + 
        str(norm_var[1]) + "\n-------------------------------------------------------")
    
    if len(independent_variables) > 1:
        vif = [variance_inflation_factor(X.values, p) for p in range(X.shape[1])]
        vif_df = pd.DataFrame({'vif': vif[1:]}, index=num_df.columns).T 
        multicoll_test = "There is a low to moderate multicollinearity in the dataset ✅" if (np.average(vif_df) >=1 and np.average(vif_df) <=5) else "There is a  high multicollinearity in the dataset ❌"
        print(str(multicoll_test) + "\n")
        with pd.option_context('expand_frame_repr', False): #hotfix to print VIF df on one line
            print(vif_df)
        print("\n-------------------------------------------------------")


    
#Linear_Regression(df,'CO2 Emissions', numerical_without_emissions, 0.05, log_model=True)   #logarithmic multiple regression
Linear_Regression(df,'CO2 Emissions', numerical_without_emissions, 0.05)  #multiple linear regression
#Linear_Regression(df,'CO2 Emissions', ['Renewable Energy Consumption'], 0.05)   #simple linear regression

In [ ]:
#determine which type of non-linear regression to use
s = 0
sns.set()
for i in numerical_without_emissions:

    plt.scatter(df['CO2 Emissions'], df[i])
    
    plt.xlabel(i)
    plt.show()

In [ ]:
def dummies(vars, df):  #given a categorical variable, adds columns corresponding to n-1 categories; one category will be used as a reference and will not be in the dataframe
    for var in vars:
        temp = pd.get_dummies(df[var], drop_first = True)
        df = pd.concat([df, temp], axis=1)
        df.drop([var], axis=1, inplace=True)
    return df

dummy_df = dummies(['Income Group', 'Region'], df) 
independent_var_dummy = list(dummy_df.columns[2:])  #making a list of the new independent variables


#MULTIPLE LINEAR REGRESSION WITH THE DUMMIES

Linear_Regression(dummy_df, 'CO2 Emissions', independent_var_dummy, 0.05)

In [ ]:
#the following function performs a polynomial regression and tests its Gauss-Markov Theorem assumptions

def PolynomialRegression(attribute, df, alpha):
    ndf = pd.DataFrame(data = df, columns = ['Renewable Energy Consumption'])
    s = np.array(ndf[attribute])
    ndf['Sq2Attr'] = s*s
    #ndf['Sq3Attr'] = s*s*s
    X = sm.add_constant(ndf)

    model = sm.OLS(df['CO2 Emissions'], (X))
    results = model.fit()
    residuals = results.resid

    #GMT assumptions
    err_mean_test = "The expectation of residuals is 0 ✓" if (scipy.stats.ttest_1samp(residuals,0)[1] > alpha) else "Residuals have a mean different than 0 ✕"
    corr_test = "Errors are not correlated ✓" if (durbin_watson(residuals) >= 1.5 and durbin_watson(residuals) <= 2.5) else "Errors are correlated ✕"
    hom_test = "Errors are homoscedastic ✓" if (sms.het_breuschpagan(residuals, results.model.exog)[1] > alpha) else "Heteroskedasticity is present ✕"
    norm_test = "Errors follow a normal distribution ✓" if(kstest(residuals, 'norm')[1] > alpha) else "Errors do not follow a normal distribution ✕"

    if (results.f_pvalue < alpha):
        print("A simple linear regression between the dependent variable CO2 Emissions and the independent variable " + str(attribute)+ " resulted in a statistically significant R2 of " + str(results.rsquared) + " with a p-value of " + str(results.f_pvalue))
    else:
        print("A simple linear regression between the dependent variable CO2 Emissions and the independent variable " + str(attribute)+ " resulted in a statistically insignificant R2 of " + str(results.rsquared) + " with a p-value of " + str(results.f_pvalue))

    print("\n" + err_mean_test + " \n" + corr_test + " \n" + hom_test + " \n" + norm_test)
    print("\n" + str(results.summary()) + "\n ---------------------------------------------------------------------------------")

PolynomialRegression('Renewable Energy Consumption', df, 0.05)
